In [9]:
%matplotlib widget

In [10]:
import camelot
from pathlib import Path
import pandas as pd
import numpy as np

In [11]:
payroll_report_pdf = Path.cwd().parent / "data" / "raw" / "payroll_reports" / "december_2022.pdf"

In [12]:
raw_tables = camelot.read_pdf(str(payroll_report_pdf), pages="all", flavor="lattice")

/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (559.18, 561.67504) does not lie in column range (45.11043823685006, 558.8473518742444)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (559.3, 561.79504) does not lie in column range (45.11043823685006, 559.0873035066506)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (559.3, 561.79504) does not lie in column range (45.11067256461608, 559.0873035066506)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (559.3, 561.90352) does not lie in column range (45.11067256461608, 558.9673276904475)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (395.47, 397.96504000000004) does not lie in column range (45.11043823685006, 395.20033857315605)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/camelot/utils.py:740: UserWarning:   (399.6

In [13]:
tables = list(map(lambda t: t.df.copy(), raw_tables))

In [14]:
def basic_cleaning(df: pd.DataFrame):
    df = df.dropna(how="all", axis=1)
    df = df.replace(r"[\r\n\s]+", " ", regex=True) \
        .replace('-', '', regex=True) \
        .replace("", np.nan)
    print("Load Table:", df.shape, "->", df.shape)
    return df

In [15]:
cln_tables = list(map(basic_cleaning, tables))

Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (9, 16) -> (9, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (10, 16) -> (10, 16)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (9, 9) -> (9, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 9) -> (10, 9)
Load Table: (10, 18) -> (10, 18)
Load Table: (10, 18) -> (10, 18)
Load Table: (10, 18) -> (10, 18)
Load Table: (10, 18) -> (

In [16]:
def is_monthly(df: pd.DataFrame):
    return df.iloc[0, :].str.contains("\w{3,9}\s\d{4}", regex=True).any()

In [17]:
def is_epf(df: pd.DataFrame):
    return df.iloc[:, 1].str.lower().str.contains('new EPF subscriber', na=False, case=False).any()

In [18]:
monthly_epf_tables = list(filter(lambda t: is_monthly(t) and is_epf(t), cln_tables))

In [19]:
def parse_date_head(df: pd.DataFrame) -> pd.Series:
    return pd.to_datetime(
        df.iloc[:, 0],
        format="%B %Y",
        errors="coerce"
    ).dropna()

In [20]:
def strip_month_headline(df: pd.DataFrame):
    date_df = parse_date_head(df)
    date_row_idx = date_df.index[0]
    return df.iloc[date_row_idx + 1:].reset_index(drop=True)


In [21]:
def exclude_totals_row(df: pd.DataFrame):
    return df[~df.iloc[:, 0].str.contains("total", na=False, case=False)]

In [22]:
def epf_correct_camelot(df: pd.DataFrame):
    # TODO: find a method to resolve this issue with camelot itself.
    df.iloc[7, [0,1]] = df.iloc[:, 0].str.extract(r'(.*) (\d[\d,.]*)$').iloc[7]
    df.iloc[2, [0,1]] = df.iloc[:, 1].str.extract(r'(.*) (\d[\d,.]*)$').iloc[2]
    return df

In [23]:
def prep_row_labels(df: pd.DataFrame):
    headings = df.iloc[:, 0].str.replace(r"(\d{2})(\d{2})", r"\1-\2", regex=True)
    headings[0:2] = ["head", "gender"]
    headings = headings.str.lower()
    return df.rename(index=headings).drop(0, axis=1)

In [89]:
def reshape_epf(df: pd.DataFrame):
    df.iloc[0] = df.iloc[0].str.replace("\n", "").replace("", np.nan).ffill()
    df = df.T.melt(id_vars=["head", "gender"], var_name="age")
    categorical_columns = ["head", "gender", "age"]
    df[categorical_columns] = df[categorical_columns].astype("category")
    # TODO: convert values to integer values
    # df.value = pd.to_numeric(df.value.str.replace(",", "").str.strip(), errors="coerce", downcast="unsigned")
    return df

In [90]:
df = monthly_epf_tables[0].pipe(strip_month_headline).pipe(exclude_totals_row).pipe(epf_correct_camelot).pipe(prep_row_labels).pipe(reshape_epf)
df

,head,gender,age,value
0,Number of new EPF subscribers during the month,Male,less than 18,6475.0
1,Number of new EPF subscribers during the month,Female,less than 18,1358.0
2,Number of new EPF subscribers during the month,Others,less than 18,NaN
3,Number of new EPF subscribers during the month,Not Available,less than 18,NaN
4,Number of new EPF subscribers during the month,Total,less than 18,7833.0
...,...,...,...,...
85,Number of exited members who rejoined and resu...,Male,more than 35,310385.0
86,Number of exited members who rejoined and resu...,Female,more than 35,73044.0
87,Number of exited members who rejoined and resu...,Others,more than 35,3.0
88,Number of exited members who rejoined and resu...,Not Available,more than 35,1.0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   head    90 non-null     category
 1   gender  90 non-null     category
 2   age     90 non-null     category
 3   value   78 non-null     object  
dtypes: category(3), object(1)
memory usage: 1.6+ KB


In [27]:
parse_date_head(monthly_epf_tables[0]).iloc[0]

Timestamp('2022-04-01 00:00:00')

In [28]:
def epf_data_pipeline(df: pd.DataFrame):
    month = parse_date_head(df).iloc[0]
    df = df.pipe(strip_month_headline) \
        .pipe(exclude_totals_row) \
        .pipe(epf_correct_camelot) \
        .pipe(prep_row_labels) \
        .pipe(reshape_epf)
    df["year"] = month.year
    df["month"] = month.month
    return df

In [99]:
fnl_epf = pd.concat(map(epf_data_pipeline, monthly_epf_tables))

In [30]:
def is_esic(df: pd.DataFrame):
    return df.iloc[:, 5].str.lower().str.contains('Number of newly registered', na=False, case=False).any()

In [101]:
monthly_esic_tables = list(filter(lambda x: is_esic(x) and is_monthly(x), tables))

In [114]:
monthly_esic_tables[0].pipe(strip_month_headline).pipe(exclude_totals_row).pipe(prep_row_labels).pipe(reshape_epf).head()

,head,gender,age,value
0,Number of existing employees who paid during t...,Male,less than 18,"4,483"
1,Number of existing employees who paid during t...,Female,less than 18,"2,803"
2,Number of existing employees who paid during t...,Others,less than 18,
3,Number of existing employees who paid during t...,Total,less than 18,"7,286"
4,Number of newly registered employees & paying ...,Male,less than 18,"1,445"


In [111]:
def esic_data_pipeline(df: pd.DataFrame):
    month = parse_date_head(df).iloc[0]
    df = df.pipe(strip_month_headline) \
        .pipe(exclude_totals_row) \
        .pipe(prep_row_labels) \
        .pipe(reshape_epf)
    df["year"] = month.year
    df["month"] = month.month
    return df

In [115]:
fnl_esic = pd.concat(map(esic_data_pipeline, monthly_esic_tables))

In [116]:
fnl_esic

,head,gender,age,value,year,month
0,Number of existing employees who paid during t...,Male,less than 18,"4,483",2022,4
1,Number of existing employees who paid during t...,Female,less than 18,"2,803",2022,4
2,Number of existing employees who paid during t...,Others,less than 18,,2022,4
3,Number of existing employees who paid during t...,Total,less than 18,"7,286",2022,4
4,Number of newly registered employees & paying ...,Male,less than 18,"1,445",2022,4
...,...,...,...,...,...,...
43,Number of existing employees who paid during t...,Total,more than 35,"1,05,64,641",2022,12
44,Number of newly registered employees & paying ...,Male,more than 35,"2,01,668",2022,12
45,Number of newly registered employees & paying ...,Female,more than 35,"68,303",2022,12
46,Number of newly registered employees & paying ...,Others,more than 35,8,2022,12
